In [1]:
#initialize a empty dictoinary to store the sentiment and the count
file_dict = {"Baseline":["output/model/0/baseline_1/",
                           "output/model/1/baseline_1/",
                           "output/model/2/baseline_1/",
                           "output/model/3/baseline_1/",
                           "output/model/4/baseline_1/"],
             "Positive":["output/model/0/Sentiment/Positive/constrained_eps_32_batch_8/",
                           "output/model/1/Sentiment/Positive/constrained_eps_32_batch_8/",
                           "output/model/2/Sentiment/Positive/constrained_eps_32_batch_8/",
                           "output/model/3/Sentiment/Positive/constrained_eps_32_batch_8/",
                           "output/model/4/Sentiment/Positive/constrained_eps_32_batch_8/"],
                "Negative":["output/model/0/Sentiment/Negative/constrained_eps_32_batch_8/",
                           "output/model/1/Sentiment/Negative/constrained_eps_32_batch_8/",
                           "output/model/2/Sentiment/Negative/constrained_eps_32_batch_8/",
                           "output/model/3/Sentiment/Negative/constrained_eps_32_batch_8/",
                           "output/model/4/Sentiment/Negative/constrained_eps_32_batch_8/"],
                "Neutral":["output/model/0/Sentiment/Neutral/constrained_eps_32_batch_8/",
                           "output/model/1/Sentiment/Neutral/constrained_eps_32_batch_8/",
                           "output/model/2/Sentiment/Neutral/constrained_eps_32_batch_8/",
                           "output/model/3/Sentiment/Neutral/constrained_eps_32_batch_8/",
                           "output/model/4/Sentiment/Neutral/constrained_eps_32_batch_8/"],
                "Formal":["output/model/0/Formality/Formal/constrained_eps_32_batch_8/",
                           "output/model/1/Formality/Formal/constrained_eps_32_batch_8/",
                           "output/model/2/Formality/Formal/constrained_eps_32_batch_8/",
                           "output/model/3/Formality/Formal/constrained_eps_32_batch_8/",
                           "output/model/4/Formality/Formal/constrained_eps_32_batch_8/"],
                "Informal":["output/model/0/Formality/Informal/constrained_eps_32_batch_8/",
                           "output/model/1/Formality/Informal/constrained_eps_32_batch_8/",
                           "output/model/2/Formality/Informal/constrained_eps_32_batch_8/",
                           "output/model/3/Formality/Informal/constrained_eps_32_batch_8/",
                           "output/model/4/Formality/Informal/constrained_eps_32_batch_8/"],
                "English":["output/model/0/Language/English/constrained_eps_32_batch_8/",
                           "output/model/1/Language/English/constrained_eps_32_batch_8/",
                           "output/model/2/Language/English/constrained_eps_32_batch_8/",
                           "output/model/3/Language/English/constrained_eps_32_batch_8/",
                           "output/model/4/Language/English/constrained_eps_32_batch_8/"],
                "French":["output/model/0/Language/French/constrained_eps_32_batch_8/",
                           "output/model/1/Language/French/constrained_eps_32_batch_8/",
                           "output/model/2/Language/French/constrained_eps_32_batch_8/",
                           "output/model/3/Language/French/constrained_eps_32_batch_8/",
                           "output/model/4/Language/French/constrained_eps_32_batch_8/"],    
                "Spanish":["output/model/0/Language/Spanish/constrained_eps_32_batch_8/",
                           "output/model/1/Language/Spanish/constrained_eps_32_batch_8/",
                           "output/model/2/Language/Spanish/constrained_eps_32_batch_8/",
                           "output/model/3/Language/Spanish/constrained_eps_32_batch_8/",
                           "output/model/4/Language/Spanish/constrained_eps_32_batch_8/"],  
                "Left":["output/model/0/Politics/Left/constrained_eps_32_batch_8/",
                           "output/model/1/Politics/Left/constrained_eps_32_batch_8/",
                           "output/model/2/Politics/Left/constrained_eps_32_batch_8/",
                           "output/model/3/Politics/Left/constrained_eps_32_batch_8/",
                           "output/model/4/Politics/Left/constrained_eps_32_batch_8/"],  
                "Right":["output/model/0/Politics/Right/constrained_eps_32_batch_8/",
                           "output/model/1/Politics/Right/constrained_eps_32_batch_8/",
                           "output/model/2/Politics/Right/constrained_eps_32_batch_8/",
                           "output/model/3/Politics/Right/constrained_eps_32_batch_8/",
                           "output/model/4/Politics/Right/constrained_eps_32_batch_8/"],                            
              }


modal_name="llava"
if modal_name == 'minigpt4':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", modal_name) for path in paths]
elif modal_name == 'llava':
    for category, paths in file_dict.items():
        file_dict[category] = [path.replace("model", modal_name) for path in paths]


# Binary classfication

In [15]:
import json

def get_classification_result(file_path):
    # Initialize an empty list to store the extracted outputs
    total_length=0
    output_list = []
    # Open the JSON file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON line
            json_line = json.loads(line)

            # Check if 'output' key exists in the JSON object
            if 'output' in json_line:
                # Add the value of 'output' key to the outputs list
                output_list.append(json_line['output'])
    count=0
    #iterate through the list of instructions and print the sentiment
    for o in output_list:
        if "yes" in o.lower():
            count+=1
            total_length+=1
        elif "no" in o.lower():
            total_length+=1
    # print(file_path,count)
    return count/total_length


classification_dict={}
#iterate through the dictionary and calculate the result
for key in file_dict:
    result_count = []
    for file in file_dict[key]:
        result=get_classification_result(file+"content_classification_result.jsonl")
        result_count.append(result)
    classification_dict[key]=round(sum(result_count)/len(result_count),2)

In [16]:
classification_dict["sentiment"]=(classification_dict["Positive"]+ classification_dict["Negative"]+ classification_dict["Neutral"])/3
classification_dict["formality"]=(classification_dict["Formal"]+ classification_dict["Informal"])/2
classification_dict["language"]=(classification_dict["English"]+ classification_dict["French"]+ classification_dict["Spanish"])/3
classification_dict["politics"]=(classification_dict["Left"]+ classification_dict["Right"])/2
classification_dict

{'Baseline': 0.43,
 'Positive': 0.69,
 'Negative': 0.43,
 'Neutral': 0.53,
 'Formal': 0.47,
 'Informal': 0.46,
 'English': 0.65,
 'French': 0.35,
 'Spanish': 0.12,
 'Left': 0.53,
 'Right': 0.63,
 'sentiment': 0.5499999999999999,
 'formality': 0.46499999999999997,
 'language': 0.37333333333333335,
 'politics': 0.5800000000000001}

In [17]:
classification_dict
#save the dictionary to a json file
import json
with open('output/aggregated_result/content_perservation/classification/minigpt4_binary_32.json', 'w') as fp:
    json.dump(classification_dict, fp)

# Similarity between oringinal and perturbed image
## LLaVA

In [1]:
import argparse
from PIL import Image
import torch
import matplotlib.pyplot as plt

def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--model_path", type=str, default="./ckpts/llava_llama_2_13b_chat_freeze")
    parser.add_argument("--gpu_id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--model_base", type=str, default=None)
    args = parser.parse_args(args=['--gpu_id', '0'])
    return args

# ========================================
#             Model Initialization
# ========================================
print('>>> Initializing Models')
from llava_llama_2.utils import get_model
args = parse_args()
print('model = ', args.model_path)
tokenizer, model, image_processor, model_name = get_model(args)
model.eval()
print('[Initialization Finished]\n')

>>> Initializing Models


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
You are using a model of type llava to instantiate a model of type llava_llama_2. This is not supported for all configurations of models and can yield errors.


model =  ./ckpts/llava_llama_2_13b_chat_freeze
llava_llama_2_13b_chat_freeze


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[Initialization Finished]



## Minigpt4

In [15]:
import argparse
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn
# import gradio as gr
from PIL import Image

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry

# imports modules for registration
from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *

from minigpt_utils import prompt_wrapper, generator

def parse_args():
    parser = argparse.ArgumentParser(description="Demo")

    parser.add_argument("--cfg-path", default="eval_configs/minigpt4_eval.yaml", help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--mode", type=str, default='VisualChatBot',
                        choices=[ "TextOnly", "VisualChatBot" ],
                        help="Inference Mode: TextOnly: Text model only (Vicuna) \n VisualChatBot: Vision model + Text model (MiniGPT4) ")
    parser.add_argument("--image_file", type=str, default='./image.bmp',
                        help="Image file")
    parser.add_argument("--output_file", type=str, default='./result.jsonl',
                        help="Output file.")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args(args=['--cfg-path', 'eval_configs/minigpt4_eval.yaml', 
                               '--gpu-id', '0', '--mode', 'VisualChatBot', '--image_file', 'clean_images/0.png', '--output_file', './result.jsonl'])
    # args = parser.parse_args()
    return args

def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True

# ========================================
#             Model Initialization
# ========================================

print('>>> Initializing Models')
args = parse_args()
cfg = Config(args)
model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
print('Initialization Finished')
my_generator = generator.Generator(model=model)

>>> Initializing Models
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading LLAMA Done
Load BLIP2-LLM Checkpoint: ckpts/pretrained_minigpt4.pth
Initialization Finished


In [2]:
from PIL import Image
from pytorch_msssim import ssim
from torchvision import transforms
import torch
import matplotlib.pyplot as plt

def denormalize(images):
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()
    images = images * std[None, :, None, None]
    images = images + mean[None, :, None, None]
    return images

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return image

def calculate_similarity(image1, image2, metric, model):
    if metric=='llava':
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        if not isinstance(image1, torch.Tensor):
            image1 = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
            image2 = image_processor.preprocess(image2, return_tensors='pt')['pixel_values'].cuda()
        emb_image1 = model.encode_images(image1.half())
        emb_image2 = model.encode_images(image2.half())
        cos_sim = cos(emb_image1.view(-1).to(torch.float32), emb_image2.view(-1).to(torch.float32))
        return cos_sim
    elif metric=='minigpt4':
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        if not isinstance(image1, torch.Tensor):
            image1 = [vis_processor(image1).unsqueeze(0).to(model.device)]
            image2 = [vis_processor(image2).unsqueeze(0).to(model.device)]
        else:
            image1=[image1]
            image2=[image2]
        prompt1 = prompt_wrapper.Prompt(model=model, img_prompts=[image1])
        emb_image1 = prompt1.img_embs[0][0]
        prompt2 = prompt_wrapper.Prompt(model=model, img_prompts=[image2])
        emb_image2 = prompt2.img_embs[0][0]
        cos_sim = cos(emb_image1.view(-1).to(torch.float32), emb_image2.view(-1).to(torch.float32))
        return cos_sim
    elif metric=='ssim':
        if not isinstance(image1, torch.Tensor):
            image1 = image_processor.preprocess(image1, return_tensors='pt')['pixel_values'].cuda()
            image2 = image_processor.preprocess(image2, return_tensors='pt')['pixel_values'].cuda()
        ssim_val = ssim(image1, image2, data_range=1, size_average=True)  
        return ssim_val

def calculate_transformations_sim(image, metric, model, show_plots=True):

    transformations = [
        transforms.GaussianBlur(kernel_size=9, sigma=(1.0, 5.0)),
        transforms.RandomAffine(degrees=45),  # Rotate by 45 degrees
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
        transforms.RandomHorizontalFlip(p=1),
        transforms.RandomPerspective(distortion_scale=0.5, p=1)
    ]
    
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()
    if show_plots:
        fig, axs = plt.subplots(1, len(transformations) + 1, figsize=(15, 10))  # +1 for the original image
        axs[0].imshow(denormalize(image_tensor)[0].detach().cpu().permute(1, 2, 0))
        axs[0].set_title('Original')
        axs[0].axis('off')

    sim_list = []
    for idx, transform in enumerate(transformations):
        transformed_image_tensor = transform(image_tensor)  # Apply transformation
        if show_plots:
            axs[idx + 1].imshow(denormalize(transformed_image_tensor)[0].detach().cpu().permute(1, 2, 0))
            axs[idx + 1].set_title(transform.__class__.__name__)
            axs[idx + 1].axis('off')
        if metric=='ssim':
            sim = calculate_similarity(denormalize(image_tensor), denormalize(transformed_image_tensor),model, metric)
        else:
            sim = calculate_similarity(denormalize(image_tensor), denormalize(transformed_image_tensor),model, metric)
        sim_list.append(sim)
    if show_plots:
        plt.show()
    return sum(sim_list)/len(sim_list)

In [3]:
sim_dict={}
sim_metric='llava'
temp_list=[]
for i in range(0,4):
    image_file1="clean_images/" + str(i) + ".png"
    image1 = load_image(image_file1)
    for j in range(i+1,5):
        image_file2="clean_images/" + str(j) + ".png"
        image2 = load_image(image_file2)
        sim =calculate_similarity(image1, image2, sim_metric, model)
        temp_list.append(sim)
sim_dict['random pairs']=round(torch.mean(torch.stack(temp_list)).item(),3)

for key in file_dict:
    if key=="Baseline":
        continue
    temp_list=[]
    for file in file_dict[key]:
        image_file1=file+"clean_prompt.bmp"
        image1 = load_image(image_file1)
        image_file2=file+"bad_prompt.bmp"
        image2 = load_image(image_file2)
        sim =calculate_similarity(image1, image2, sim_metric,model)
        temp_list.append(sim)
    #add key and the mean of the cosine similarity to a dictionary
    sim_dict[key]=round(torch.mean(torch.stack(temp_list)).item(),3)

# Call the function with the dummy_loader and the list of transformations
temp_list=[]
for i in range(0,5):
    image_file="clean_images/" + str(i) + ".png"
    image = load_image(image_file)
    temp_list.append(calculate_transformations_sim(image, model, sim_metric, False))
sim_dict['augmentations']=round(torch.mean(torch.stack(temp_list)).item(),3)

print(sim_dict)

NameError: name 'file_dict' is not defined

In [51]:
sim_dict["sentiment"]=(sim_dict["Positive"]+ sim_dict["Negative"]+ sim_dict["Neutral"])/3
sim_dict["formality"]=(sim_dict["Formal"]+ sim_dict["Informal"])/2
sim_dict["language"]=(sim_dict["English"]+ sim_dict["French"]+ sim_dict["Spanish"])/3
sim_dict["politics"]=(sim_dict["Left"]+ sim_dict["Right"])/2
sim_dict


{'random pairs': 0.535,
 'Positive': 0.641,
 'Negative': 0.639,
 'Neutral': 0.571,
 'Formal': 0.682,
 'Informal': 0.605,
 'English': 0.612,
 'French': 0.695,
 'Spanish': 0.713,
 'Left': 0.616,
 'Right': 0.582,
 'augmentations': 0.809,
 'sentiment': 0.617,
 'formality': 0.6435,
 'language': 0.6733333333333333,
 'politics': 0.599}

In [46]:
sim_dict


{'random pairs': 0.535,
 'Positive': 0.641,
 'Negative': 0.639,
 'Neutral': 0.571,
 'Formal': 0.682,
 'Informal': 0.605,
 'English': 0.612,
 'French': 0.695,
 'Spanish': 0.713,
 'Left': 0.616,
 'Right': 0.582}

In [52]:
# embedding_dict
#save the dictionary to a json file
import json
with open('output/aggregated_result/content_perservation/embedding/minigpt4_minigpt_result_32.json', 'w') as fp:
    json.dump(sim_dict, fp)

In [5]:
jailbreak_dict = {"llava":["/home/tz362/Desktop/projects/Visual-Adversarial-Examples-Jailbreak-Large-Language-Models/results_llava_llama_v2_constrained_32_image_2/"],
                "minigpt4":["/home/tz362/Desktop/projects/Visual-Adversarial-Examples-Jailbreak-Large-Language-Models/visual_constrained_eps_32/"],                            
                }
sim_metric='llava'
sim_dict = {}
for key in jailbreak_dict:
    temp_list=[]
    for file in jailbreak_dict[key]:
        image_file1="/home/tz362/Desktop/projects/Visual-Adversarial-Examples-Jailbreak-Large-Language-Models/adversarial_images/clean_2.png"
        image1 = load_image(image_file1)
        image_file2=file+"bad_prompt_temp_5000.bmp"
        image2 = load_image(image_file2)
        sim =calculate_similarity(image1, image2, sim_metric,model)
        temp_list.append(sim)
    #add key and the mean of the cosine similarity to a dictionary
    sim_dict[key]=round(torch.mean(torch.stack(temp_list)).item(),3)
sim_dict

{'llava': 0.311, 'minigpt4': 0.22}